### Harmonize survey results

#### Seperates the survey data into object counts, uses location and date as survey identifier.


Fetches data from the local source and esyablishes the following variables:

1. dutch_codes
2. swiss_codes
3. dutch_surveys
4. swiss_surveys
5. swiss_beaches


Establishes directory variables for fetching and putting to all subdirectories:

1. data
2. beaches
3. codes
4. geo
5. output

provides a script to update the remote data.

In [1]:
# sys things
import os
import sys
import json

# networks
import requests

# data
import pandas as pd
import numpy as np
import scipy
import math
import seaborn as sns

import resources.utilities.utility_functions as ut

In [2]:
# get folder extesions
data, beaches, codes, geo, output=ut.make_local_paths()
print("look for resources here\n")
print(data, beaches, codes, geo, output)

look for resources here

resources/surveydata resources/locationdata resources/mlwcodedefs resources/geodata output


In [3]:
project_directory = ut.make_project_folder(output, "harmonized_results")
print("put stuff here:\n")
print(project_directory)

put stuff here:

output/harmonized_results


In [4]:
# survey_data
dutch_surveys = pd.read_csv(data+'/dataset_macrolitter_NL.csv')

# use the aggregated hd data. This accounts for the custom codes used in Switzerland
swiss_surveys = pd.read_csv(data+'/aggregated_hd_surveys.csv')

# location data
swiss_beaches = pd.read_csv(beaches+'/hammerdirt_beaches.csv')

In [5]:
print("Columns from cleaned up dutch data\n")
print(dutch_surveys.columns)

print("\nColumns from cleaned up swiss data\n")
print(swiss_surveys.columns)

Columns from cleaned up dutch data

Index(['Unnamed: 0', 'Gebiedscode', 'Rivier', 'Coordinaten', 'Lat', 'Lon',
       'meting', 'date', 'doy', 'dop',
       ...
       'sanitair_tissues_wc_papier', 'sanitair_toiletverfrissers',
       'sanitair_overig_sanitair', 'medisch_verpakkingen', 'medisch_spuiten',
       'medisch_overig_medisch', 'Unnamed: 121', 'Totaal', 'Items/km',
       'granulaat_korrels'],
      dtype='object', length=125)

Columns from cleaned up swiss data

Index(['loc_date', 'code', 'date', 'pcs_m', 'quantity', 'location',
       'water_name'],
      dtype='object')


In [6]:
print("the first ten rows are admin data\n")
print(dutch_surveys.columns[:10])
print("\nthe next 115 is code data\n")
print(dutch_surveys.columns[10:])
print("\none row is equal to one survey")

the first ten rows are admin data

Index(['Unnamed: 0', 'Gebiedscode', 'Rivier', 'Coordinaten', 'Lat', 'Lon',
       'meting', 'date', 'doy', 'dop'],
      dtype='object')

the next 115 is code data

Index(['plastic_6_packringen', 'plastic_tassen',
       'plastic_kleine_plastic_tasjes',
       'plastic_drankflessen_groterdan_halveliter',
       'plastic_drankflessen_kleinerdan_halveliter',
       'plastic_wikkels_van_drankflessen',
       'plastic_verpakking_van_schoonmaakmiddelen',
       'plastic_voedselverpakkingen_frietbakjes_etc',
       'plastic_cosmeticaverpakkingen',
       'plastic_motorolieverpakking_groterdan50cm',
       ...
       'sanitair_tissues_wc_papier', 'sanitair_toiletverfrissers',
       'sanitair_overig_sanitair', 'medisch_verpakkingen', 'medisch_spuiten',
       'medisch_overig_medisch', 'Unnamed: 121', 'Totaal', 'Items/km',
       'granulaat_korrels'],
      dtype='object', length=115)

one row is equal to one survey


In [7]:
print("This is the number of rows\n")
print(len(dutch_surveys))
print("\nThis is the number of unique 'Gebiedscode' values\n")
print(len(dutch_surveys.Gebiedscode.unique()))

This is the number of rows

391

This is the number of unique 'Gebiedscode' values

212


In [8]:
# key the Gebiedscode to an int
count_gebied = {n:i+1 for i,n in enumerate(dutch_surveys.Gebiedscode.unique())}

# grab only the object count columns and the river
obj_counts = dutch_surveys.columns[10:-4]

In [9]:
# these columns are admin data
admin = ['Gebiedscode', 'Rivier', 'Coordinaten', 'Lat', 'Lon', 'meting', 'date', 'doy', 'dop']
date = ['date']

# unique id per row:
loc_date = ['Gebiedscode', 'date']

# these are no longer needed
no_longer_needed = ['Unnamed: 121', 'Totaal', 'Items/km', 'granulaat_korrels']



def make_location_groups(x, keys):
    try:
        name = keys[x]
    except:
        name = x
    return name

# make a column of the keys created for Gebiedscode
dutch_surveys['name'] = dutch_surveys.Gebiedscode.map(lambda x: make_location_groups(x, count_gebied))

# zip that up with the date
dutch_surveys['loc_date'] = list(zip(dutch_surveys.name, dutch_surveys.date))

# make an admin df:
dutch_admin = dutch_surveys[['name','loc_date', *admin]]
a_file_name = F"{project_directory}/dutch_admin_h.csv"
dutch_admin.to_csv(a_file_name)

print("This is the admin data plus a name tag and a loc_date tag\n")
dutch_admin.iloc[:5]

This is the admin data plus a name tag and a loc_date tag



,name,loc_date,Gebiedscode,Rivier,Coordinaten,Lat,Lon,meting,date,doy,dop
0,1,"(1, 20171021)",W(078a)R-ZHNL,Waal,#REF!,51.8227,5.0159,0,20171021,294.0,293.0
1,2,"(2, 20171103)",W(074b)R-REFE,Waal,#REF!,51.8290,5.0805,0,20171103,307.0,306.0
2,3,"(3, 20171016)",W(062a)L-GENL,Waal,#REF!,51.8126,5.2358,0,20171016,289.0,288.0
3,4,"(4, 20171004)",W(048a)L-GENL,Waal,#REF!,51.8475,5.4159,0,20171004,277.0,276.0
4,5,"(5, 20171002)",W(046a)L-GENL,Waal,#REF!,51.8588,5.4222,0,20171002,275.0,274.0


In [10]:
print("Thi)
dutch_admin.iloc[0]

name                       1
loc_date       (1, 20171021)
Gebiedscode    W(078a)R-ZHNL
Rivier                  Waal
Coordinaten            #REF!
Lat                  51.8227
Lon                   5.0159
meting                     0
date                20171021
doy                      294
dop                      293
Name: 0, dtype: object

In [11]:
# a place to store the results
new_results=[]

def seperate_results(x, a_list):
    # alist is the list of columns to keep
    # return x because we don't want to chage the df
    for element in a_list:
        stuff = {'name':x['name'], 'date':x['date'], 'loc_date':x['loc_date'], 'river':x['Rivier'], 'object':element, 'quantity':x[element] }
        new_results.append(stuff)
    return x

# apply

dutch_surveys.apply(lambda row: seperate_results(row, obj_counts), axis=1)

# make new df 
a_results_df = pd.DataFrame(new_results)
a_file_name = F"{project_directory}/dutch_surveys_h.csv"
a_results_df.to_csv(a_file_name)

print()
print("This is the object results with a  name tag and a loc_date tag\n")
a_results_df.iloc[:10]


This is the object results with a  name tag and a loc_date tag



,name,date,loc_date,river,object,quantity
0,1,20171021,"(1, 20171021)",Waal,plastic_6_packringen,0.0
1,1,20171021,"(1, 20171021)",Waal,plastic_tassen,0.0
2,1,20171021,"(1, 20171021)",Waal,plastic_kleine_plastic_tasjes,2.0
3,1,20171021,"(1, 20171021)",Waal,plastic_drankflessen_groterdan_halveliter,0.0
4,1,20171021,"(1, 20171021)",Waal,plastic_drankflessen_kleinerdan_halveliter,1.0
5,1,20171021,"(1, 20171021)",Waal,plastic_wikkels_van_drankflessen,3.0
6,1,20171021,"(1, 20171021)",Waal,plastic_verpakking_van_schoonmaakmiddelen,0.0
7,1,20171021,"(1, 20171021)",Waal,plastic_voedselverpakkingen_frietbakjes_etc,6.0
8,1,20171021,"(1, 20171021)",Waal,plastic_cosmeticaverpakkingen,0.0
9,1,20171021,"(1, 20171021)",Waal,plastic_motorolieverpakking_groterdan50cm,0.0


In [12]:
print("The total by loc_date\n")
print(a_results_df.groupby('loc_date').quantity.sum())

The total by loc_date

loc_date
(1, 20171021)      170.0
(1, 20180318)      139.0
(1, 20181021)      228.0
(1, 20190225)      158.0
(2, 20171103)      207.0
                   ...  
(208, 20190316)    209.0
(209, 0)           410.0
(210, 20190226)    946.0
(211, 20190303)    754.0
(212, 20190306)    263.0
Name: quantity, Length: 389, dtype: float64


In [13]:
print("The total by river\n")
print(a_results_df.groupby('river').quantity.sum())

The total by river

river
Maas         113812.0
Nederrijn      7279.0
Waal          31324.0
Name: quantity, dtype: float64


In [14]:
print("The total quantity\n")
print(a_results_df.quantity.sum())

The total quantity

152415.0


In [15]:
tops = a_results_df.groupby('object').quantity.sum()
tops = tops.sort_values(ascending=False)

print("The top ten objects:\n")
print(tops[:10])

The top ten objects:

object
plastic_plastic_stukjes_2_5_50cm_zacht_plastic    24273.0
plastic_plastic_stukjes_0_2_5cm_zacht_plastic     18085.0
plastic_piepschuim_0_2_5cm                        15036.0
plastic_piepschuim_2_5_50cm                       10762.0
plastic_snoep_snack_chipsverpakking               10622.0
plastic_plastic_stukjes_0_2_5cm_hard_plastic       9314.0
plastic_plastic_stukjes_2_5_50cm_hard_plastic      7619.0
plastic_doppen_en_deksels                          6937.0
plastic_voedselverpakkingen_frietbakjes_etc        3673.0
sanitair_plastic_wattenstaafjes                    3258.0
Name: quantity, dtype: float64


### Change the object definitions to OSPAR codes

The objects definitions need to changed to OSPAR

In [16]:
## !!! refresh the data from the hammerdirt api here:

# a = requests.get('https://mwshovel.pythonanywhere.com/api/surveys/daily-totals/code-totals/swiss/')
# b = requests.get('https://mwshovel.pythonanywhere.com/api/list-of-beaches/swiss/')
# c = requests.get('https://mwshovel.pythonanywhere.com/api/mlw-codes/list/')

# # the surveys need to be unpacked:
# swiss_surveys = ut.unpack_survey_results(a.json())
# swiss_surveys = pd.DataFrame(swiss_surveys)

# # adding location date column
# swiss_surveys['loc_date'] = list(zip(swiss_surveys['location'], swiss_surveys['date']))

# # hold the original
# x = a.json()

# print("survey columns")
# print(swiss_surveys.columns)

# swiss_beaches = pd.DataFrame(b.json())
# print("beach columns")
# print(swiss_beaches.columns)

# print("code columns")
# swiss_codes = pd.DataFrame(c.json())
# print(swiss_codes.columns)

# swiss_surveys.to_csv(data+'/hammerdirt_data.csv')
# swiss_beaches.to_csv(beaches+'/hammerdirt_beaches.csv')
# swiss_codes.to_csv(codes+'/swiss_codes.csv')
